<a href="https://colab.research.google.com/github/czarinadamienne/GRP6_CUDAPROJECT/blob/main/GRP6_CUDAPROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing **Histogram Counting** and using **Shared memory concept** in CUDA

### Check CUDA

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb 18 05:33:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## (1) C Implementation

In [1]:
%%writefile C_histcount.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <stdint.h>

//histogram counting
void histcount(size_t n, int32_t* histbins, int32_t* vec) {
    int index = 0;
    for (int i = 0; i < n; i++) {
        index = vec[i] % 10;
        histbins[index]++;
    }
}

int main(int argc, char** argv) {
    const size_t size = 1 << 28; //change to 1 << 28
    const size_t arr_bytes = size * sizeof(int32_t);
    const size_t hist_bytes = 10 * sizeof(int32_t);

    const size_t repeat = 30;

    //dynamically allocate
    int32_t* vec, *histbins;
    vec = (int32_t*)malloc(arr_bytes);
	histbins = (int32_t*)malloc(hist_bytes);

	for (int i = 0; i < 10; i++) {
		histbins[i] = 0;
	}

    //time test
    clock_t start, end;

    //initialize arrays with index value
    for (int i = 0; i < size; i++) {
        vec[i] = (int32_t) i;
    }

    //avoid cache miss
    histcount(size, histbins, vec);
    for (int i = 0; i < 10; i++) { //clear the histogram
        histbins[i] = 0;
    }

    //timer
    double elapse, time_taken;
    elapse = 0.0f;

    for (int i = 0; i < repeat; i++) {
        start = clock();
        histcount(size, histbins, vec);
        end = clock();
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse += time_taken;

        if (i < repeat - 1) { //clear the histogram
            for (int j = 0; j < 10; j++) {
                histbins[j] = 0;
            }
        }
    }

    printf("Historgram Bins: \n");
    for (int i = 0; i < 10; i++) {
        printf("Historgram Bin #%d: %d\n", i + 1, histbins[i]);
    }

    printf("\n\nC function:\n");
    printf("Average execution time: %f milliseconds\n", elapse / repeat);
    printf("Number of runs: %zu\n", repeat);
    printf("Array size: %zu\n", size);

    int32_t histcheck[10] = { 0 };
    for (int i = 0; i < size; i++) {
        histcheck[vec[i] % 10]++;
    }

    /*printf("\n\nVector: ");
    for (int i = 0; i < size; i++){
      printf("%d", vec[i]);
    }*/

    //error checker
    size_t error = 0;
    for (int i = 0; i < 10; i++) {
        if (histbins[i] != histcheck[i]) {
            error++;
        }
    }

    printf("\nNumber of wrong bins in C program: %zu\n\n", error);

    //free memory
    free(vec);
	  free(histbins);

    return 0;
}

Writing C_histcount.c


In [2]:
%%shell
gcc C_histcount.c -o C_histcount
./C_histcount

Historgram Bins: 
Historgram Bin #1: 26843546
Historgram Bin #2: 26843546
Historgram Bin #3: 26843546
Historgram Bin #4: 26843546
Historgram Bin #5: 26843546
Historgram Bin #6: 26843546
Historgram Bin #7: 26843545
Historgram Bin #8: 26843545
Historgram Bin #9: 26843545
Historgram Bin #10: 26843545


C function:
Average execution time: 1345.293433 milliseconds
Number of runs: 30
Array size: 268435456

Number of wrong bins in C program: 0



## (2) CUDA Implementation with Shared Memory Concept

In [7]:
%%writefile CUDA_histcount.cu
#include <stdio.h>
#include <stdlib.h>

//CUDA histcount kernel using shared memory
__global__ void histcount(size_t n, int32_t* histbins, int32_t* vec) {
    int ind = 0;
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    __shared__ int32_t sharedHist[10]; // Static shared memory

    // Initialize shared memory with 0
    for (int i = threadIdx.x; i < 10; i += blockDim.x) {
      sharedHist[i] = 0;
    }

    __syncthreads(); // Syncrhonize the threads to avoid race conditions

    for (int i = index; i < n; i += stride) {
        ind = vec[i] % 10;
        atomicAdd(&sharedHist[ind], 1);
    }

    __syncthreads();

    // Write back from shared memory to global memory
    for (int i = threadIdx.x; i < 10; i += blockDim.x) {
      atomicAdd(&histbins[i], sharedHist[i]);
    }
}

int main() {
    const size_t size = 1 << 28; //change to 1 << 28
    const size_t arr_bytes = size * sizeof(int32_t);
    const size_t hist_bytes = 10 * sizeof(int32_t);

    const size_t repeat = 30;

    //cuda allocate cpu and gpu memory
    int32_t *vec, *histbins;
    cudaMallocManaged(&vec, arr_bytes);
    cudaMallocManaged(&histbins, hist_bytes);

	//initialize histogram bins to 0
    cudaMemset(histbins, 0, hist_bytes);

    //get gpu in
    int device = -1;
    cudaGetDevice(&device);

    //mem advise
    cudaMemAdvise(vec, arr_bytes, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(vec, arr_bytes, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


    //prefetch to create CPU page memory
    cudaMemPrefetchAsync(vec, arr_bytes, cudaCpuDeviceId, NULL);

    //prefetch to create GPU page memory
    cudaMemPrefetchAsync(histbins, hist_bytes, device, NULL);

    //initialize arrays with index value
    for (int i = 0; i < size; i++) {
        vec[i] = (int32_t)i;
    }

    //prefetching CPU-GPU
    cudaMemPrefetchAsync(vec, arr_bytes, device, NULL);
	  cudaMemPrefetchAsync(histbins, hist_bytes, device, NULL);

    //cuda kernel
    size_t threads = 1024;
    size_t blocks = (size + threads - 1) / threads;

    for (int i = 0; i < repeat; i++) {
        histcount <<<blocks, threads>>> (size, histbins, vec);

        if (i < repeat - 1) { //clear the histogram
            cudaMemset(histbins, 0, hist_bytes);
        }
    }

    cudaDeviceSynchronize(); //wait GPU to finish

    //prefetch from gpu-cpu
    cudaMemPrefetchAsync(histbins, hist_bytes, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(vec, arr_bytes, cudaCpuDeviceId, NULL);

    printf("Historgram Bins: \n");
    for (int i = 0; i < 10; i++) {
        printf("Historgram Bin #%d: %d\n", i + 1, histbins[i]);
    }

    printf("\n\nCUDA kernel:\n");
    printf("Number of blocks: %lu\n", blocks);
    printf("Number of threads: %lu\n", threads);
    printf("Number of runs: %lu\n", repeat);
    printf("Array size: %lu\n\n", size);

    int indcheck = 0;
    int32_t histcheck[10] = { 0 };
    for (int i = 0; i < size; i++) {
        indcheck = vec[i] % 10;
        histcheck[indcheck]++;
    }

    //error checker
    size_t error = 0;
    for (int i = 0; i < 10; i++) {
        if (histbins[i] != histcheck[i]) {
            error++;
        }
    }

    printf("Number of errors in CUDA program: %zu\n\n", error);

    //free memory
    cudaFree(vec);
    cudaFree(histbins);

    return 0;
}

Overwriting CUDA_histcount.cu


In [6]:
%%shell
nvcc -o CUDA_histcount CUDA_histcount.cu -arch=sm_75
nvprof ./CUDA_histcount

==959== NVPROF is profiling process 959, command: ./CUDA_histcount
Historgram Bins: 
Historgram Bin #1: 26843546
Historgram Bin #2: 26843546
Historgram Bin #3: 26843546
Historgram Bin #4: 26843546
Historgram Bin #5: 26843546
Historgram Bin #6: 26843546
Historgram Bin #7: 26843545
Historgram Bin #8: 26843545
Historgram Bin #9: 26843545
Historgram Bin #10: 26843545


CUDA kernel:
Number of blocks: 1048576
Number of threads: 256
Number of runs: 30
Array size: 268435456

Number of errors in CUDA program: 0

==959== Profiling application: ./CUDA_histcount
==959== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.89%  225.76ms        30  7.5253ms  6.3050ms  11.172ms  histcount(unsigned long, int*, int*)
                    0.11%  247.20us        30  8.2390us  1.9200us  182.01us  [CUDA memset]
      API calls:   46.35%  332.33ms         6  55.388ms  23.513us  231.06ms  cudaMemPrefetchAsync
                   30.67%  219.9

In [8]:
%%shell
nvcc -o CUDA_histcount CUDA_histcount.cu -arch=sm_75
nvprof ./CUDA_histcount

==1077== NVPROF is profiling process 1077, command: ./CUDA_histcount
Historgram Bins: 
Historgram Bin #1: 26843546
Historgram Bin #2: 26843546
Historgram Bin #3: 26843546
Historgram Bin #4: 26843546
Historgram Bin #5: 26843546
Historgram Bin #6: 26843546
Historgram Bin #7: 26843545
Historgram Bin #8: 26843545
Historgram Bin #9: 26843545
Historgram Bin #10: 26843545


CUDA kernel:
Number of blocks: 262144
Number of threads: 1024
Number of runs: 30
Array size: 268435456

Number of errors in CUDA program: 0

==1077== Profiling application: ./CUDA_histcount
==1077== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.92%  296.44ms        30  9.8812ms  7.0141ms  14.143ms  histcount(unsigned long, int*, int*)
                    0.08%  223.55us        30  7.4510us  1.7280us  157.27us  [CUDA memset]
      API calls:   42.46%  347.45ms         6  57.908ms  21.829us  239.66ms  cudaMemPrefetchAsync
                   35.42%  2